In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Can be done using one for loop
import zipfile

with zipfile.ZipFile("/kaggle/input/carvana-image-masking-challenge/train.zip") as z:
    z.extractall(".")
with zipfile.ZipFile("/kaggle/input/carvana-image-masking-challenge/train_masks.zip") as z:
    z.extractall(".")
with zipfile.ZipFile("/kaggle/input/carvana-image-masking-challenge/train_masks.csv.zip") as z:
    z.extractall(".")
    

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import random
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from PIL import Image
# from torchvision.transforms import functional as F

In [ ]:
len(os.listdir('/kaggle/working/train'))

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# for i in os.listdir('/kaggle/working/train'):
#very interesting thing. I always use random number fron numpy and use for loop. Use random.choice
i = random.choice(os.listdir('/kaggle/working/train'))
image = os.path.join('/kaggle/working/train',i)
im = cv2.imread(image)
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
# plt.imshow(im)
file = i.replace('.jpg','_mask.gif')
mask = os.path.join('/kaggle/working/train_masks',file)
msk = Image.open(mask).convert('L')
# plt.imshow(msk)
fig,axes = plt.subplots(1,2,figsize=(12,6))
axes[0].imshow(im)
axes[1].imshow(msk)

    # print(i)
    # break

In [ ]:
i = random.choice(os.listdir('/kaggle/working/train'))
image = os.path.join('/kaggle/working/train',i)
plt.imshow(np.array(Image.open(image).convert('RGB')))

In [ ]:
plt.imshow(np.array(Image.open(image).convert('RGB'))/255.0)

In [ ]:
df = pd.read_csv('/kaggle/working/train_masks.csv')

In [ ]:
df.head()

Next steps:
1. Dataset class
2. Create model
3. Train model

In [ ]:
class CarvanaDataset(Dataset):
    def __init__(self,image_dir,mask_dir,transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(self.image_dir)

    def __len__(self):
        # return len(os.listdir(self.image_dir))
        return len(self.images)

    def __getitem__(self,idx):
        img_path = os.path.join(self.image_dir,self.images[idx])
        mask_path = os.path.join(self.mask_dir,self.images[idx].replace('.jpg','_mask.gif'))
        image = np.array(Image.open(img_path).convert('RGB'),dtype=np.float32)/255.0
        mask = np.array(Image.open(mask_path).convert('L'),dtype=np.float32)/255.0
        # mask = mask/255.0 #Normalize mask to 0 and 1
        mask = np.expand_dims(mask, axis=-1)

        if self.transform:
            augmentations = self.transform(image=image,mask=mask)
            image = augmentations['image']
            mask = augmentations['mask']
            mask = mask.permute(2, 0, 1)
        else:
            image = torch.tensor(image,dtype = torch.float32).permute(2,0,1)
            mask = torch.tensor(mask,dtype = torch.float32).unsqueeze(0)
            
        return image,mask
        
        
    

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_func = A.Compose([A.Resize(256,256),
                       A.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5)),
                       ToTensorV2()])

In [ ]:
transform_func

In [ ]:
len(os.listdir('/kaggle/working/train'))

In [ ]:
os.chdir('/kaggle/working/')
list = ['train1_images','train1_masks','test1_images','test1_masks']
for i in list:
    os.makedirs(os.path.join('/kaggle/working/',i),exist_ok=True)


In [ ]:
import shutil
k = 0
for i in (os.listdir('/kaggle/working/train')):
    if k < 4500:
        dest = (os.path.join('/kaggle/working/train',i))
        src = '/kaggle/working/train1_images'
        shutil.copy(dest,src)
        mask_dest = (os.path.join('/kaggle/working/train_masks',i.replace('.jpg','_mask.gif')))
        mask_src = '/kaggle/working/train1_masks'
        shutil.copy(mask_dest,mask_src)
    else:
        dest = (os.path.join('/kaggle/working/train',i))
        src = '/kaggle/working/test1_images'
        shutil.copy(dest,src)
        mask_dest = (os.path.join('/kaggle/working/train_masks',i.replace('.jpg','_mask.gif')))
        mask_src = '/kaggle/working/test1_masks'
        shutil.copy(mask_dest,mask_src)
    k+=1
        
    # break
    

In [ ]:
image_dir='/kaggle/working/train'
mask_dir='/kaggle/working/train_masks'
train_data = CarvanaDataset(image_dir,mask_dir,transform=transform_func)

In [ ]:
train_data[0][1].shape

In [ ]:
# fig,ax = plt.subplots(1,2,figsize=(12,6))
# # plt.imshow(train_data[1][1])
# ax[0].imshow(train_data[100][0])
# ax[1].imshow(train_data[100][1])


In [ ]:
# image_dir='/kaggle/working/train'
# mask_dir='/kaggle/working/train_masks'
train_data = CarvanaDataset(image_dir='/kaggle/working/train1_images',mask_dir='/kaggle/working/train1_masks',transform=transform_func)
test_data = CarvanaDataset(image_dir='/kaggle/working/test1_images',mask_dir='/kaggle/working/test1_masks',transform=transform_func)

In [ ]:
train_loader = DataLoader(train_data,batch_size=4,num_workers=1,shuffle=True)
test_loader = DataLoader(test_data,batch_size=4,num_workers=1,shuffle=False)

In [ ]:
train_data[0][1].shape

Model

In [ ]:

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        
        self.encoder1 = DoubleConv(in_channels, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.bottleneck = DoubleConv(512, 1024)
        
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)
        
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        
        bottleneck = self.bottleneck(self.pool(enc4))
        
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((enc4, dec4), dim=1)
        dec4 = self.decoder4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((enc3, dec3), dim=1)
        dec3 = self.decoder3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((enc2, dec2), dim=1)
        dec2 = self.decoder2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((enc1, dec1), dim=1)
        dec1 = self.decoder1(dec1)
        
        return self.final_conv(dec1)
    

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = UNet().to(device)

In [ ]:
sample_input = torch.randn(1, 3, 256, 256)  # Batch size of 1, 3 channels, 256x256 image
output = model(sample_input.to(device))
print(output.shape)  # Output shape should be [1, 1, 256, 256]


In [ ]:
#Hyperparam
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-04)

In [ ]:
from tqdm import tqdm

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, masks in tqdm(train_loader):
        images = images.to(device)  # Move to GPU if available
        masks = masks.to(device)
        
        # Forward pass
        outputs = model(images)
        # outputs=outputs.squeeze(0)
        loss = loss_fn(outputs, masks)
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.4f}")
    
    # Scheduler step (optional)
    # scheduler.step(train_loss)


In [ ]:
torch.save(model.state_dict(), 'carvana_unet_model.pth')

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True